### Notebook to prototype REDCap API calls

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import seaborn as sns

In [2]:
def api_call(url, query, logger=None):
    
    # headers = requests.utils.default_headers()
    # headers.update(
    #     {
    #         'User-Agent': 'My User Agent 1.0',
    #     }
    # )

    r = requests.post(url, data=query, verify=False)
    http_status = str(r.status_code)
    print(f'HTTP Status: {http_status}')

    if http_status == "200":
        query_results = r.json()
        query_df = pd.DataFrame(query_results)

    else:
        print(f"RedCap API request Failed with HTTP Status: {http_status}")

    return query_df

In [3]:
DATASET_ROOT = "/home/nikhil/projects/Parkinsons/qpn/"

# Current nipoppy manifest
release_dir = f"{DATASET_ROOT}/releases/"
current_release = "Jan_2024"
current_nipoppy_manifest_csv = f"{release_dir}{current_release}/tabular/manifest.csv"
legacy_qpn_imaging_codes_xlsx = f"{release_dir}{current_release}/tabular/assessments/QPN_Imaging_Codes.xlsx"

# Legacy DoB
legacy_participants_DOB = f"{DATASET_ROOT}/tabular/legacy_freeze/QPN-DOB-90subjects.csv"
legacy_participants_DOB_codes = f"{DATASET_ROOT}/tabular/legacy_freeze/QPN-DOB-codes.csv"

# redcap_report_csv = f"{release_dir}{current_release}/tabular/assessments/COPN-MRIDataReport2023110_DATA_LABELS_2024-01-10_1004.csv"
redcap_report_csv = f"{release_dir}{current_release}/tabular/assessments/COPN-MRIDataReport2023110_DATA_LABELS_2024-02-19_0851.csv"

MRI_dates_csv = f"{DATASET_ROOT}/scratch/mri_dates_sanity_check.csv"
MRI_acq_data_csv = f"{release_dir}{current_release}/tabular/MRI_acqdata.csv"


### Manifest

In [4]:
visit_list = ["MRI_v1"] 
session_id_list = ["01"]
session_list = [f"ses-{idx}" for idx in session_id_list]

manifest_cols = ["participant_id", "visit", "session"]

current_nipoppy_manifest_df = pd.read_csv(current_nipoppy_manifest_csv)
current_nipoppy_manifest_df = current_nipoppy_manifest_df[current_nipoppy_manifest_df["visit"].isin(visit_list)]
current_nipoppy_manifest_df = current_nipoppy_manifest_df[current_nipoppy_manifest_df["session"].isin(session_list)]
current_nipoppy_manifest_df = current_nipoppy_manifest_df[manifest_cols]
current_nipoppy_manifest_df["participant_id"] = current_nipoppy_manifest_df["participant_id"].str.upper()
nipoppy_participants = current_nipoppy_manifest_df["participant_id"].unique()
n_participants = len(nipoppy_participants)
print(f"n_participants: {n_participants}")
current_nipoppy_manifest_df.head()

n_participants: 298


,participant_id,visit,session
0,PD00016,MRI_v1,ses-01
1,PD00020,MRI_v1,ses-01
2,PD00032,MRI_v1,ses-01
3,PD00048,MRI_v1,ses-01
4,PD00119,MRI_v1,ses-01


### Legacy demographics and visit dates
- Using this while REDCap is getting updated

In [5]:
sheet_name = "recruit_manifest"
usecols = ["participant_id", "group", "sex", "dob", "visit_01"]
legacy_recruit_df = pd.read_excel(legacy_qpn_imaging_codes_xlsx, sheet_name=sheet_name, engine='openpyxl', usecols=usecols)
legacy_recruit_df["participant_id"] = legacy_recruit_df["participant_id"].str.upper()

legacy_col_dict = {"group": "legacy_group", "sex":"legacy_sex", "dob":"legacy_dob", "visit_01": "legacy_date_MRI"}
legacy_recruit_df = legacy_recruit_df.rename(columns=legacy_col_dict)

legacy_participants = legacy_recruit_df["participant_id"].unique()
n_legacy_participants = len(legacy_participants)
print(f"n_legacy_participants: {n_legacy_participants}")

legacy_recruit_df["legacy_dob"] = pd.to_datetime(legacy_recruit_df["legacy_dob"], errors="coerce", dayfirst=True)
legacy_recruit_df["legacy_date_MRI"] = pd.to_datetime(legacy_recruit_df["legacy_date_MRI"], errors="coerce", dayfirst=True)

# calculate age at MRI_V01
legacy_recruit_df["legacy_age_MRI"] = legacy_recruit_df["legacy_date_MRI"] - legacy_recruit_df["legacy_dob"]
legacy_recruit_df["legacy_age_MRI"] = np.round(legacy_recruit_df["legacy_age_MRI"].dt.days / 365.25, 2)

legacy_recruit_df["visit_id"] = "MRI_v1"
legacy_recruit_df.head()

n_legacy_participants: 208


,participant_id,legacy_sex,legacy_dob,legacy_group,legacy_date_MRI,legacy_age_MRI,visit_id
0,MNI0056,M,1942-05-21,PD,2021-08-18,79.24,MRI_v1
1,MNI0058,M,1964-03-14,PD,2021-08-18,57.43,MRI_v1
2,MNI0068,M,1952-08-05,PD,2021-08-27,69.06,MRI_v1
3,MNI0079,F,1971-11-25,PD,2021-12-22,50.08,MRI_v1
4,MNI0103,M,1939-08-17,PD,2021-08-11,81.98,MRI_v1


#### Add legacy DoB

In [6]:
DOB_col = "Date of Birth/Date de naissance"
cross_ref_col = "Reference number"
record_col = "Record ID"

legacy_DOB = pd.read_csv(legacy_participants_DOB)
legacy_DOB = legacy_DOB.rename(columns={ DOB_col:"dob"})

legacy_DOB_codes = pd.read_csv(legacy_participants_DOB_codes)
legacy_DOB_codes = legacy_DOB_codes.rename(columns={record_col:"participant_id"})

legacy_DOB = pd.merge(legacy_DOB_codes, legacy_DOB, how="left", on=cross_ref_col).drop(columns=[cross_ref_col])

legacy_DOB["dob"] = pd.to_datetime(legacy_DOB["dob"], errors="coerce", dayfirst=False)
legacy_DOB.head()

,participant_id,dob
0,MNI0369,1951-01-01
1,MNI0381,1943-11-10
2,MNI0416,1952-01-11
3,MNI0420,1961-09-10
4,MNI0438,1951-04-25


#### Add MRI dates from Suivi_RPQ and dicom headers to the legacy df

In [7]:
MRI_dates_df = pd.read_csv(MRI_dates_csv)
n_mri_dates_participants = len(MRI_dates_df["participant_id"].unique())
print(f"n_mri_dates_participants: {n_mri_dates_participants}")

MRI_dates_df = MRI_dates_df[MRI_dates_df["visit_id"]=="MRI_v1"][["participant_id","visit_id","suivi_MRI_date","dicom_date"]]
MRI_dates_df["suivi_MRI_date"] = pd.to_datetime(MRI_dates_df["suivi_MRI_date"], errors="coerce", dayfirst=False)
MRI_dates_df["dicom_date"] = pd.to_datetime(MRI_dates_df["dicom_date"], errors="coerce", dayfirst=False)

legacy_recruit_df = pd.merge(MRI_dates_df, legacy_recruit_df, on=["participant_id","visit_id"], how="left")

legacy_recruit_df.head()

n_mri_dates_participants: 298


,participant_id,visit_id,suivi_MRI_date,dicom_date,legacy_sex,legacy_dob,legacy_group,legacy_date_MRI,legacy_age_MRI
0,MNI0028,MRI_v1,NaT,2023-10-04,NaN,NaT,NaN,NaT,NaN
1,MNI0056,MRI_v1,2021-08-18,2021-08-18,M,1942-05-21,PD,2021-08-18,79.24
2,MNI0058,MRI_v1,2021-08-18,2021-08-18,M,1964-03-14,PD,2021-08-18,57.43
3,MNI0068,MRI_v1,2021-08-27,2021-08-27,M,1952-08-05,PD,2021-08-27,69.06
4,MNI0079,MRI_v1,2021-12-22,2021-12-22,F,1971-11-25,PD,2021-12-22,50.08


#### Add MRI acq dates

In [8]:
MRI_acq_data_df = pd.read_csv(MRI_acq_data_csv).drop(columns=["index","participant_dicom_dir"])
MRI_acq_data_df.loc[MRI_acq_data_df["session"]=="ses-01", "visit_id"] = "MRI_v1"
MRI_acq_data_df.loc[MRI_acq_data_df["session"]=="ses-02", "visit_id"] = "MRI_v2"

legacy_recruit_df = pd.merge(MRI_acq_data_df, legacy_recruit_df, on=["participant_id","visit_id"], how="left")
legacy_recruit_df["scanner_acq_date"] = pd.to_datetime(legacy_recruit_df["scanner_acq_date"], errors="coerce", dayfirst=False)
legacy_recruit_df.head()

,participant_id,session,scanner_acq_date,visit_id,suivi_MRI_date,dicom_date,legacy_sex,legacy_dob,legacy_group,legacy_date_MRI,legacy_age_MRI
0,MNI0028,ses-01,2023-10-04,MRI_v1,NaT,2023-10-04,NaN,NaT,NaN,NaT,NaN
1,MNI0056,ses-01,2021-08-18,MRI_v1,2021-08-18,2021-08-18,M,1942-05-21,PD,2021-08-18,79.24
2,MNI0058,ses-01,2021-08-18,MRI_v1,2021-08-18,2021-08-18,M,1964-03-14,PD,2021-08-18,57.43
3,MNI0068,ses-01,2021-08-27,MRI_v1,2021-08-27,2021-08-27,M,1952-08-05,PD,2021-08-27,69.06
4,MNI0079,ses-01,2021-12-22,MRI_v1,2021-12-22,2021-12-22,F,1971-11-25,PD,2021-12-22,50.08


In [9]:
MRI_acq_data_df.head()

,participant_id,session,scanner_acq_date,visit_id
0,MNI0028,ses-01,2023-10-04,MRI_v1
1,MNI0056,ses-01,2021-08-18,MRI_v1
2,MNI0058,ses-01,2021-08-18,MRI_v1
3,MNI0068,ses-01,2021-08-27,MRI_v1
4,MNI0079,ses-01,2021-12-22,MRI_v1


### Redcap config
QPN specific reports:
['QPN participants', 'External QPN (June 2021)', 'Demographic QPN', 'QPN-Clinical questionnaire', 'QPN sex', 'Diagnosis QPN', 'MoCA-MDS-UPDRS part 3', 'Victoria - Weston Project', 'MotorAndNon-Motor', 'MoCA']

In [10]:
redcap_config_json = f"{DATASET_ROOT}/proc/.redcap.json"
redcap_config = json.load(open(redcap_config_json))
url = redcap_config["url"]
redcap_reports = list(redcap_config["queries"].keys())
n_redcap_reports = len(redcap_reports)
print(f"redcap_reports ({n_redcap_reports}): {redcap_reports}")

redcap_reports (12): ['QPN participants', 'External QPN (June 2021)', 'Demographic QPN', 'QPN-Clinical questionnaire', 'QPN sex', 'Diagnosis QPN', 'MoCA-MDS-UPDRS part 3', 'Victoria - Weston Project', 'MotorAndNon-Motor', 'MoCA', 'C-OPN/QPN - OMEGA - Neuropsy', 'QPN MoCA-UPDRS-Neuropsy data_Sarah']


In [11]:
query_dict = {}
redcap_participants = []
api_report_list = ["Demographic QPN","Victoria - Weston Project","QPN MoCA-UPDRS-Neuropsy data_Sarah"]
for query_label in redcap_reports:

    if query_label in api_report_list:
        query = redcap_config["queries"][query_label]

        # run query
        print(f"Running query {query_label}...")
        query_df = api_call(url, query, logger=None)
        query_df["record_id"] = query_df["record_id"].str.upper()

        # get the list of participants
        _participants = query_df["record_id"].unique()
        redcap_participants.extend(_participants)
        n_participants = len(redcap_participants)

        # get the list of redcap events
        redcap_events = query_df["redcap_event_name"].unique()
        n_events = len(redcap_events)

        print(f"Fetched {n_participants} participants and {n_events} event_ids: {redcap_events}")

        query_dict[query_label] = query_df.copy()

redcap_participants = list(set(redcap_participants))

Running query Demographic QPN...


NameError: name 'headers' is not defined

### Redcap report
This comes from Sarah with DoB

In [ ]:
redcap_report_df = pd.read_csv(redcap_report_csv)

demo_col_dict = {"Record ID:": "participant_id", "Event Name": "redcap_event_name", 
                 "Enrolment Group:": "group", "Date of Birth":"dob", "1. Sex": "sex", 
                 "Date of MoCA administration": "moca_date","MDS-UPDRS Date":"updrs_date",
                 "Neuropsycholgical Test Date:": "neuropsy_date"}

redcap_report_df = redcap_report_df[list(demo_col_dict.keys())].rename(columns=demo_col_dict)

redcap_report_df["participant_id"] = redcap_report_df["participant_id"].str.upper()
redcap_report_df["sex"] = redcap_report_df["sex"].replace({"Male/Masculin":"M", "Female/Féminin":"F"})

redcap_report_participants = redcap_report_df["participant_id"].unique()
n_participants = redcap_report_df["participant_id"].nunique()
n_events = redcap_report_df["redcap_event_name"].unique()
n_DOBs = redcap_report_df[redcap_report_df['dob'].notnull()]["participant_id"].nunique()

print(f"Number of participants: {n_participants}")
print(f"Number of events: {n_events}")

print(f"Number of available DOB: {n_DOBs}")
redcap_report_df.head()

### Add dob from legacy data

In [ ]:
redcap_report_df[(redcap_report_df["participant_id"].isin(legacy_dob_participants)) & 
                 (redcap_report_df["dob"].isna()) & 
                 (redcap_report_df["redcap_event_name"] == "Baseline (Arm 1: C-OPN)")]

In [ ]:
legacy_dob_participants = legacy_DOB["participant_id"].unique()
legacy_dob_participants = set(legacy_dob_participants) & set(redcap_report_df[redcap_report_df["dob"].isna()]["participant_id"])

print(f"Number of legacy participants with DOB: {len(legacy_dob_participants)}")
      
redcap_report_df["dob"] = pd.to_datetime(redcap_report_df["dob"], errors="coerce", dayfirst=False)

for p in legacy_dob_participants:
    redcap_report_df.loc[(redcap_report_df["participant_id"] == p) & 
                         (redcap_report_df["redcap_event_name"] == "Baseline (Arm 1: C-OPN)"), 
                         "dob"] = legacy_DOB.loc[legacy_DOB["participant_id"]==p,"dob"].values[0]
    
n_DOBs = redcap_report_df[redcap_report_df['dob'].notnull()]["participant_id"].nunique()
print(f"Number of available DOB: {n_DOBs}")

In [ ]:
redcap_report_df[redcap_report_df["participant_id"] == p]

#### Calculate ages from dates in the redcap report

In [ ]:

redcap_report_df["moca_date"] = pd.to_datetime(redcap_report_df["moca_date"], errors="coerce", dayfirst=False)
redcap_report_df["updrs_date"] = pd.to_datetime(redcap_report_df["updrs_date"], errors="coerce", dayfirst=False)
redcap_report_df["neuropsy_date"] = pd.to_datetime(redcap_report_df["neuropsy_date"], errors="coerce", dayfirst=False)

## fill in dob for followup visits
dob_df = redcap_report_df[["participant_id","dob"]].copy()
# dob_df = dob_df.dropna(subset="dob")
redcap_report_df = pd.merge(redcap_report_df.drop(columns="dob"), dob_df, on="participant_id")

redcap_report_df["age_moca"] = redcap_report_df["moca_date"] - redcap_report_df["dob"]
redcap_report_df["age_moca"] = np.round(redcap_report_df["age_moca"].dt.days / 365.25, 2)

redcap_report_df["age_updrs"] = redcap_report_df["updrs_date"] - redcap_report_df["dob"]
redcap_report_df["age_updrs"] = np.round(redcap_report_df["age_updrs"].dt.days / 365.25, 2)

redcap_report_df["age_neuropsy"] = redcap_report_df["neuropsy_date"] - redcap_report_df["dob"]
redcap_report_df["age_neuropsy"] = np.round(redcap_report_df["age_neuropsy"].dt.days / 365.25, 2)

redcap_report_participants = redcap_report_df["participant_id"].unique()
print(f"n_redcap_report_df: {len(redcap_report_participants)}")

redcap_report_df.head()

### Participant tallys

In [ ]:
# redcap_participants = [p.upper() for p in redcap_participants]

print(f"Number of nipoppy participants: {len(nipoppy_participants)}")
print(f"Number of legacy participants: {len(legacy_participants)}")
print(f"Number of redcap participants: {len(redcap_participants)}")
print(f"Number of redcap report participants: {len(redcap_report_participants)}")


a = set(nipoppy_participants)
b = set(legacy_participants)
c = set(redcap_participants)
d = set(redcap_report_participants)

nipoppy_legacy_common_participants = a.intersection(b)
n_nipoppy_legacy_common_participants = len(nipoppy_legacy_common_participants)

nipoppy_redcap_common_participants = a.intersection(c)
n_nipoppy_redcap_common_participants = len(nipoppy_redcap_common_participants)

nipoppy_redcap_report_common_participants = a.intersection(d)
n_nipoppy_redcap_report_common_participants = len(nipoppy_redcap_report_common_participants)

nipoppy_not_in_legacy_participants  = a.difference(b) # nipoppy participants not in the legacy spreadsheets
n_nipoppy_not_in_legacy_participants = len(nipoppy_not_in_legacy_participants)

nipoppy_not_in_redcap_participants  = a.difference(c) # nipoppy participants not in the redcap yet
n_nipoppy_not_in_redcap_participants = len(nipoppy_not_in_redcap_participants)

nipoppy_not_in_redcap_report_participants  = a.difference(d) # nipoppy participants not in the redcap report yet
n_nipoppy_not_in_redcap_report_participants = len(nipoppy_not_in_redcap_report_participants)

missing_nipoppy_participants = a.difference(b).difference(c).difference(d)
n_missing_nipoppy_participants = len(missing_nipoppy_participants)

new_redcap_participants  = nipoppy_redcap_common_participants.difference(d) # nipoppy-redcap participants not in the report yet
n_new_redcap_participants = len(new_redcap_participants)

print(f"nipoppy-legacy common participants: {n_nipoppy_legacy_common_participants}")
print(f"nipoppy-redcap common participants: {n_nipoppy_redcap_common_participants}")
print(f"nipoppy-recdap_report common participants: {n_nipoppy_redcap_report_common_participants}")

print(f"nipoppy-redcap participants not in the report yet (n={n_new_redcap_participants}): {new_redcap_participants}")
print(f"missing_nipoppy_participants (n={n_missing_nipoppy_participants}): {missing_nipoppy_participants}")

### Demographics, Dx, and summary clinical scores
- `Demographic QPN`
    - "study_visit_age", "gender", "yrs_education"
        - Note: study visit age will be different for different assessments
        - **study_visit_age is not useful since it's self reported at random times**
- `Diagnosis QPN` does not have Dx for all participants
- `Victoria - Weston Project` (Legend for Determined diagnosis)
    - If score = 0, Parkinson's Disease (PD)  
    - If score = 1, Progressive Supranuclear Palsy (PSP)  
    - If score = 2, Multiple System Atrophy (MSA) 
    - If score = 3, Corticobasal Syndrome (CBS)  
    - If score = 4, Dementia wi1th Lewy Bodies (DLB)  
    - If score = 5, Frontotemporal Dementia (FTD)  
    - If score = 6, Essential Tremor (ET)  
    - If score = 7, REM Sleep Behaviour Disorder (RBD)
- `MotorAndNon-Motor` 
    - summary clinical scores (updrs, moca)
    - verify moca column with extra point i.e. `moca_extra_point`
- `QPN MoCA-UPDRS-Neuropsy data_Sarah`
    - Has additional updrs variables 
    





### Aggregate useful redcap data

In [ ]:
index_columns = ["record_id", "redcap_event_name"]

demo_cols = ["gender", "yrs_education"]
demo_df = query_dict['Demographic QPN'][index_columns + demo_cols].copy()

dx_cols = ["diagnosis_determined", "duration_disease"]
dx_df = query_dict['Victoria - Weston Project'][index_columns + dx_cols]

diagnosis_determined_label_map = {
    "0" : "PD",
    "1" : "PSP",
    "2" : "MSA",
    "3" : "CBS", 
    "4" : "DLB",
    "5" : "FTD", 
    "6" : "ET",
    "7" : "RBD"
}
dx_df.loc[:,"diagnosis_determined"] = dx_df["diagnosis_determined"].astype(str).replace(diagnosis_determined_label_map).copy()

updrs_cols = ["mds_updrs_h_y", "updrs_score_part_1", "updrs_score_part_2", "updrs_score_part_3", "updrs_score_part_4"]
moca_cols = ['moca_result'] 
moca_subscore_cols = ['moca_result_2', 'moca_result_3', 'moca_result_4','moca_result_5', 'moca_result_6', 
                      'moca_result_7', 'moca_result_8','moca_result_9']

score_cols = updrs_cols + moca_cols
score_df = query_dict["QPN MoCA-UPDRS-Neuropsy data_Sarah"][index_columns + score_cols].copy()

redcap_df = pd.merge(demo_df, dx_df, on=index_columns)
redcap_df = pd.merge(score_df, redcap_df, on=index_columns, how="left")

redcap_df["gender"] = redcap_df["gender"].replace({"Male/Masculin":"M", "Female/Féminin":"F"})

n_redcap_common_participants = len(redcap_df["record_id"].unique())
print(f"Found {n_redcap_common_participants} recdap-nipoppy common participants")

print(f"redcap events: {redcap_df['redcap_event_name'].unique()}")

redcap_df.head()

### Merge MRI, legacy and redcap tabular data

In [ ]:
# merge redcap query df
redcap_event_list = ['Baseline (Arm 1: C-OPN)',
                     '12 Months Follow-Up/Suivi (Arm 1: C-OPN)', 
                     '18 Months Follow-Up/Suivi (Arm 1: C-OPN)',
                     '24 Months Follow-Up/Suivi (Arm 1: C-OPN)']

nipoppy_redcap_df = redcap_df[(redcap_df["record_id"].str.upper().isin(nipoppy_participants)) & 
                              (redcap_df["redcap_event_name"].isin(redcap_event_list))].copy()

n_nipoppy_redcap_participants = len(nipoppy_redcap_df["record_id"].unique())
print(f"n_nipoppy_redcap_participants: {n_nipoppy_redcap_participants}")
               
nipoppy_redcap_df = nipoppy_redcap_df.replace("", np.nan)
nipoppy_redcap_df = nipoppy_redcap_df.rename(columns={"record_id": "participant_id"})

nipoppy_redcap_df = pd.merge(nipoppy_redcap_df, legacy_recruit_df, on="participant_id", how="left")

n_nipoppy_redcap_participants = len(nipoppy_redcap_df["participant_id"].unique())
print(f"n_nipoppy_redcap_participants: {n_nipoppy_redcap_participants}")

redcap_events = nipoppy_redcap_df["redcap_event_name"].unique()
print(f"redcap_events: {redcap_events}")

nipoppy_redcap_df.head()

In [ ]:
nipoppy_redcap_df[nipoppy_redcap_df["participant_id"]=="PD01253"]

### Merge redcap_report_df with nipoppy_redcap df

In [ ]:
nipoppy_redcap_report_df = redcap_report_df[(redcap_report_df["participant_id"].str.upper().isin(nipoppy_participants)) & 
                              (redcap_report_df["redcap_event_name"].isin(redcap_event_list))].copy()

n_nipoppy_redcap_report_participants = len(nipoppy_redcap_report_df["participant_id"].unique())
print(f"n_nipoppy_redcap_participants: {n_nipoppy_redcap_report_participants}")
               
nipoppy_redcap_report_df = nipoppy_redcap_report_df.replace("", np.nan)

## Get all the age columns
nipoppy_redcap_report_df_filtered = nipoppy_redcap_report_df[["participant_id", "redcap_event_name", "group", "sex", "dob",
                                                               "moca_date",	"updrs_date", "neuropsy_date", 
                                                               "age_moca", "age_updrs", "age_neuropsy"]].copy()

nipoppy_redcap_filtered_df = pd.merge(nipoppy_redcap_report_df_filtered, nipoppy_redcap_df, 
                                      on=["participant_id","redcap_event_name"], how="left")

n_participants = nipoppy_redcap_filtered_df["participant_id"].nunique()
print(f"n_nipoppy_redcap_report_merged_participants: {n_participants}")

# calculate suivi IRM and dicom ages based on redcap DoB
nipoppy_redcap_filtered_df["suivi_age_MRI"] = nipoppy_redcap_filtered_df["suivi_MRI_date"] - nipoppy_redcap_filtered_df["dob"]
nipoppy_redcap_filtered_df["suivi_age_MRI"] = np.round(nipoppy_redcap_filtered_df["suivi_age_MRI"].dt.days / 365.25, 2)

nipoppy_redcap_filtered_df["dicom_age_MRI"] = nipoppy_redcap_filtered_df["scanner_acq_date"] - nipoppy_redcap_filtered_df["dob"]
nipoppy_redcap_filtered_df["dicom_age_MRI"] = np.round(nipoppy_redcap_filtered_df["dicom_age_MRI"].dt.days / 365.25, 2)

nipoppy_redcap_filtered_df.head(10)

### Generate bagel(s)
- neuro-bagel
- dash-bagel (currenly this is a melted version of neuro-bagel)

**Note**: QPN has different `visit` names for MRI, UPDRS, MoCA, Neuropsy etc.

Using redcap events as evidence of multiple visits. However we are NOT assuming that two assessments (e.g. UPDRS and MoCA) are co-acquired in the same redcap event. This will be inferred using dates once available. 

In [ ]:
redcap_index_cols = ["participant_id", "redcap_event_name"]
redcap_event_visit_id_dict = {"Baseline (Arm 1: C-OPN)": "v1"}

demo_cols = ['group', 'sex', 'yrs_education']
dx_cols = ['diagnosis_determined', 'duration_disease']
date_columns = ["moca_date", "updrs_date", "neuropsy_date", "suivi_MRI_date", "scanner_acq_date", "legacy_date_MRI"]
age_cols = ["legacy_age_MRI","suivi_age_MRI","dicom_age_MRI","age_moca", "age_updrs", "age_neuropsy"]

screen_cols = redcap_index_cols + demo_cols + dx_cols + date_columns + age_cols

screen_cols_rename_dict = {
    "yrs_education": "years_education_at_screening",
    # "diagnosis_determined": "diagnosis",
    "duration_disease": "duration_disease_at_screening",
    "group": "group_at_screening"
}

screen_df = nipoppy_redcap_filtered_df[screen_cols].copy()
screen_df = screen_df.rename(columns=screen_cols_rename_dict)

# ------------------------------------------------------------------------------------ #
# Check and add nipoppy participants which yet don't have any redcap data
# Only populates the participant_id and bids_id columns (required by neurobagel)
# ------------------------------------------------------------------------------------ #
nipoppy_participants_without_pheno_data_df = pd.DataFrame()
nipoppy_participants_without_pheno_data_df["participant_id"] = list(missing_nipoppy_participants)
nipoppy_participants_without_pheno_data_df["redcap_event_name"] = "missing"

print(f"appending {nipoppy_participants_without_pheno_data_df.shape[0]} participants without pheno data")
screen_df = pd.concat([screen_df, nipoppy_participants_without_pheno_data_df], axis=0)

print(f"screen_df participants: {screen_df['participant_id'].nunique()}")
# ------------------------------------------------------------------------------------ #

screen_df.head()

### Pivot the screen_df to conform to tabular demographics CSV schema
- picking dicom date as the MRI_v1 date. Need to validate these against Suivi dates

In [ ]:
def assign_assessment_visits(participant_df, id_vars, age_col_dict):

    redcap_event_months_dict = {"Baseline (Arm 1: C-OPN)": 0, "12 Months Follow-Up/Suivi (Arm 1: C-OPN)": 12,
                            "18 Months Follow-Up/Suivi (Arm 1: C-OPN)": 18, "24 Months Follow-Up/Suivi (Arm 1: C-OPN)": 24}

    if "Baseline (Arm 1: C-OPN)" in participant_df["redcap_event_name"].unique():
        participant_df_melt = participant_df.melt(id_vars=id_vars, var_name="assessment", value_name="age")
        
        # removes assessment rows with missing age (artifact of the melt operation)
        participant_df_melt = participant_df_melt.dropna(subset=["age"], how="all", axis=0)

        if len(participant_df_melt) > 0:

            # assign months_since_baseline to be able to sort the dataframe based on time
            participant_df_melt.loc[:,"months_since_baseline"] = participant_df_melt["redcap_event_name"].replace(redcap_event_months_dict)
            participant_df_melt = participant_df_melt.sort_values(["participant_id","months_since_baseline"])
            
            # drops duplicate MRI visits (artifact of merge operation from MRI dicom dates)
            participant_df_melt = participant_df_melt.drop_duplicates(subset=["participant_id","assessment","age"], keep="first")
            
            participant_df_melt["assessment"] = participant_df_melt["assessment"].replace(age_col_dict)

            # 
            new_participant_df = pd.DataFrame() # avoids messy .loc assignment to the original unsorted df 
            for assessment in participant_df_melt["assessment"].unique():
                participant_assessment_df = participant_df_melt[participant_df_melt["assessment"] == assessment].sort_values("age")
                n_visits = len(participant_assessment_df)
                # print(f"n_visits for {assessment}: {n_visits}")
                participant_assessment_df["visit_id"] = np.array(range(1, n_visits+1))
                new_participant_df = pd.concat([new_participant_df, participant_assessment_df], axis=0)

            new_participant_df = new_participant_df.drop(columns=["months_since_baseline"])   
            new_participant_df["visit_id"] = "v" + new_participant_df["visit_id"].astype(str)
 
        else:
            print(f"Participant {participant_df['participant_id'].unique()} has no assessment info")
            new_participant_df = participant_df[id_vars].copy()
    else:
        print(f"Participant {participant_df['participant_id'].unique()} has no baseline visit")
        new_participant_df = participant_df[id_vars].copy()

    return new_participant_df


In [ ]:
id_vars=["participant_id","redcap_event_name","group_at_screening","sex",
         "years_education_at_screening","duration_disease_at_screening", "diagnosis_determined"]

temporal_cols = ["medication_status"] + updrs_cols + moca_cols

age_col_dict = {"dicom_age_MRI": "MRI",
                "age_updrs": "UPDRS", 
                "age_moca": "MOCA", 
                "age_neuropsy": "NEUROPSY"}

demographic_df = pd.DataFrame()
for participant_id in screen_df["participant_id"].unique():
    participant_df = screen_df[screen_df["participant_id"] == participant_id].copy()
    participant_df = participant_df[id_vars + list(age_col_dict.keys())].copy()
    participant_df = assign_assessment_visits(participant_df, id_vars, age_col_dict)

    if participant_df is not None:
        demographic_df = pd.concat([demographic_df, participant_df], axis=0)    


demographic_df = demographic_df.sort_values(["participant_id", "visit_id"])
reorder_cols = id_vars + ["assessment", "visit_id", "age"]
demographic_df = demographic_df[reorder_cols]
demographic_df.head()

In [ ]:
print(f"demographic_df participants: {demographic_df['participant_id'].nunique()}")
print(len(demographic_df))

# demographic_df = demographic_df.drop_duplicates(subset=["assessment", "age"])

print(f"demographic_df participants: {demographic_df['participant_id'].nunique()}")
print(len(demographic_df))

In [ ]:
demographic_df.tail()

### Add Phenotype data

In [ ]:
nipoppy_redcap_df

In [ ]:
updrs_df = nipoppy_redcap_filtered_df[redcap_index_cols + updrs_cols].copy()
updrs_df["assessment"] = "UPDRS"
updrs_df["medication_status"] = np.nan

moca_df = nipoppy_redcap_filtered_df[redcap_index_cols + moca_cols].copy()
moca_df["assessment"] = "MOCA"
moca_df["medication_status"] = np.nan

In [ ]:

pheno_id_cols = ['participant_id', 'redcap_event_name','assessment', 'visit_id' ]
updrs_df = pd.merge(demographic_df[pheno_id_cols], updrs_df,
                    on=['participant_id', 'redcap_event_name', 'assessment'], how="right")
moca_df = pd.merge(demographic_df[pheno_id_cols], moca_df,
                    on=['participant_id', 'redcap_event_name', 'assessment'], how="right")

pheno_df = pd.concat([updrs_df, moca_df], axis=0)

pheno_df.head()

In [ ]:
## Filter NA values from messy merges
pheno_df = pheno_df.sort_values(["participant_id","assessment", "visit_id"])
pheno_df = pheno_df.dropna(subset="visit_id", axis=0)
pheno_df = pheno_df.drop_duplicates(keep="first")
pheno_df.head()


In [ ]:
demographic_df[demographic_df["participant_id"] == "MNI0206"]

In [ ]:
pheno_df[pheno_df["participant_id"] == "MNI0206"]

### Generate availability 
Status options: 1) "Validated" 2) "Missing" 3) "Not collected" 

In [ ]:
bagel_df = pd.merge(demographic_df, pheno_df, on=["participant_id","redcap_event_name","assessment","visit_id"], how="left") 

bagel_participants = bagel_df["participant_id"].unique()
print(f"n_bagel_participants: {len(bagel_participants)}")

bagel_demo_cols = ["age", "sex", "years_education_at_screening", "duration_disease_at_screening"]

bagel_status_cols = []
for col in updrs_cols + moca_cols + bagel_demo_cols:
    bagel_df[f"{col}_status"] = ~bagel_df[col].isna()
    bagel_status_cols.append(f"{col}_status")   
    n_available_participants = np.sum(bagel_df[f"{col}_status"])
    print(f"{col}, n_available_participants: {n_available_participants}")

print(f"bagel df shape: {bagel_df.shape}")
bagel_df.head()

### Status only bagel (no clinical info for public digest/dashboard)

In [ ]:
bagel_id_cols = ['participant_id', 'redcap_event_name', 'group_at_screening', "diagnosis_determined", "assessment", "visit_id"]
dash_bagel_df_cols = bagel_id_cols + bagel_status_cols
dash_bagel_df = bagel_df[dash_bagel_df_cols].copy()
dash_bagel_df.head()

In [ ]:
dash_bagel_df[dash_bagel_df["participant_id"]=="MNI0056"]

### dash bagel (melt)


In [ ]:
# Dashboard variables
DASH_INDEX_COLUMNS = ["participant_id", "redcap_event_name", "assessment", "visit_id"]
DASH_NAME_COL = "assessment_name"
DASH_VAL_COL = "assessment_score"

dash_df = dash_bagel_df.melt(id_vars=DASH_INDEX_COLUMNS, var_name=DASH_NAME_COL, value_name=DASH_VAL_COL)
# dash_df = dash_df.rename(columns={"visit_id": "session"})
dash_df["session"] = dash_df["assessment"] + "_" + dash_df["visit_id"]

# Remove duplicate [`participant_id` x `session`] resulting into NaN assessment score rows
dash_df = dash_df.dropna(subset=["assessment_score"], axis=0)


### save phenotypic data
- Saves `demographics.csv` --> data collected at screening i.e. age, sex, group, education etc. 
- Saves `assessments.csv` i.e. collated data from clinical assessments i.e. UPDRS, MoCA

In [ ]:
save_pheno = True

visits_csv_path = f"{release_dir}{current_release}/tabular/assessments/visits.csv"
demograph_csv_path = f"{release_dir}{current_release}/tabular/demographics/demographics.csv"
assessment_csv_path = f"{release_dir}{current_release}/tabular/assessments/assessments.csv"

print(f"demographic participants: {demographic_df['participant_id'].nunique()}")
print(f"pheno participants: {pheno_df['participant_id'].nunique()}")
if save_pheno:
    print(f"Saving data to {demograph_csv_path} and {assessment_csv_path}")
    screen_df.to_csv(visits_csv_path, index=False)
    demographic_df.to_csv(demograph_csv_path, index=False)
    pheno_df.to_csv(assessment_csv_path, index=False)
    

### save bagels

In [ ]:
save_bagels = True

bagel_csv_path = f"{release_dir}{current_release}/tabular/bagels/bagel.csv"
bagel_tsv_path = f"{release_dir}{current_release}/tabular/bagels/bagel.tsv"
dash_csv_path = f"{release_dir}{current_release}/tabular/bagels/dash_bagel.csv"
public_digest_csv_path = f"../digest/qpn_tabular_availability_digest.csv"

if save_bagels:
    bagel_df.to_csv(bagel_csv_path, index=False)
    bagel_df.to_csv(bagel_tsv_path, index=False, sep="\t")
    dash_df.to_csv(dash_csv_path, index=False)
    dash_df.to_csv(public_digest_csv_path, index=False)
    print(f"Bagel saved to {dash_csv_path}")

### Diff betweeen demo and pheno participants

In [ ]:
demo_participants = set(demographic_df['participant_id'].unique())
pheno_participants = set(pheno_df['participant_id'].unique())

print(f"demo_participants: {len(demo_participants)}")
print(f"pheno_participants: {len(pheno_participants)}")

demo_participants_without_pheno_data = list(set(demo_participants).difference(pheno_participants))
print(f"demo_participants_without_pheno_data: {len(demo_participants_without_pheno_data)}")

check_recruit_df = pd.DataFrame(data=demo_participants_without_pheno_data, columns=["record_id"],index=None)
check_recruit_df.to_csv(f"{release_dir}{current_release}/tabular/check_recruit.csv", index=False)

print(f"Saved {release_dir}{current_release}/tabular/check_recruit.csv")

In [ ]:
demographic_df[demographic_df["assessment"]=="MRI"]["participant_id"].nunique()

In [ ]:
demographic_df[demographic_df["assessment"]=="MRI"]

### Test dicom header for acq date

In [ ]:
from pydicom import dcmread
import pandas as pd
from nipoppy.workflow.dicom_org.utils import check_valid_dicom
import os 

def get_acq_date(dcm_dir):
    """ Parses dicom header to get the acquisition date
    """
    dcm_file = os.listdir(dcm_dir)[0]
    dcm_path = os.path.join(dcm_dir, dcm_file)

    if check_valid_dicom(dcm_path):
        with open(dcm_path, 'rb') as infile:
            ds = dcmread(infile)

        acq_date = ds.AcquisitionDate
        acq_date = pd.to_datetime(acq_date, format="%Y%m%d")

    else:
        acq_date = None

    return acq_date


In [ ]:
dcm_path = "/home/nikhil/projects/Parkinsons/qpn/dicom/ses-01/PD00339/"
get_acq_date(dcm_path)